# Matrix-matrix multiplication via rank-1 updates

We continue to look at how the FLAMEJulia API can be used to implement different matrix-matrix multiplication algorithms.  

First, we create some matrices.

In [1]:
m = 4
n = 3
k = 5

C = rand(m, n)
println( "C = ")
C

C = 


4×3 Array{Float64,2}:
 0.523918  0.907567  0.129295 
 0.636291  0.976137  0.0390747
 0.64707   0.838093  0.933974 
 0.839264  0.478611  0.659468 

In [2]:
Cold = copy( C ) # an alternative way of doing a "hard" copy, in this case of a matrix
    
A = rand(m, k)
println( "A = " )
A

A = 


4×5 Array{Float64,2}:
 0.894742  0.0650961  0.858902  0.567061  0.0937017
 0.13064   0.880969   0.231842  0.754032  0.485299 
 0.604483  0.740901   0.103469  0.768992  0.361006 
 0.660144  0.246472   0.679104  0.512128  0.753729 

In [3]:
B = rand(k, n)
println( "B = " )
B

B = 


5×3 Array{Float64,2}:
 0.514185  0.762752   0.394554
 0.816862  0.35512    0.914569
 0.710208  0.0968876  0.443689
 0.350955  0.840034   0.508605
 0.942621  0.572481   0.900676

## <h2>The algorithm  </h2>  <image src="https://studio.edx.org/c4x/UTAustinX/UT.5.01x/asset/Gemm_nn_unb_var3.png" alt="Matrix-matrix multiplication via rank-1 updates picture" width="80%">

<h2> The routine <code> Gemm_nn_unb_var3( A, B, C ) </code> </h2>

This routine computes $ C := A B + C $ via rank-1 updates.  The "\_nn\_" means that this is the "No transpose, No transpose" case of matrix multiplication.  
The reason for this is that the operations $ C := A^T B + C $ ("\_tn\_" or "Transpose, No transpose"), $ C := A B^T + C $ ("\_nt\_" or "No transpose, Transpose"), and $ C := A^T B^T + C $ ("\_tt\_" or "Transpose, Transpose") are also encountered.  
    
The specific laff function we will use is
<ul>
<li> <code> laff.ger!( alpha, x, y, A ) </code> which computes the rank-1 update (adds a multiple of an outer product to a matrix)
$ A := \alpha x y^T + A $. 
    </li>
</ul>

Use the <a href="https://studio.edx.org/c4x/UTAustinX/UT.5.01x/asset/index.html"> Spark webpage</a> to generate a code skeleton.  (Make sure you adjust the name of the routine.)

In [4]:
include("../flame.jl")
using .flame
include("../laff/laff.jl")
using .laff

function Gemm_nn_unb_var3!(A, B, C)

    AL, AR = flame.part_1x2(A, 
                            0, "LEFT")

    BT, 
    BB  = flame.part_2x1(B, 
                         0, "TOP")

    while size(AL, 2) < size(A, 2)

        A0, a1, A2 = flame.repart_1x2_to_1x3(AL, AR, 
                                             1, "RIGHT")

        B0,  
        b1t, 
        B2   = flame.repart_2x1_to_3x1(BT, 
                                       BB, 
                                       1, "BOTTOM")

        #------------------------------------------------------------#

        laff.ger!( 1.0, a1, b1t, C )

        #------------------------------------------------------------#

        AL, AR = flame.cont_with_1x3_to_1x2(A0, a1, A2, 
                                            "LEFT")

        BT, 
        BB  = flame.cont_with_3x1_to_2x1(B0,  
                                         b1t, 
                                         B2,  
                                         "TOP")

    end

end

Gemm_nn_unb_var3! (generic function with 1 method)

In [5]:
C = copy( Cold ) # restore C 

Gemm_nn_unb_var3!( A, B, C )

println( "C - ( Cold + A * B )" )
C - ( Cold + A * B )

C - ( Cold + A * B )


4×3 Array{Float64,2}:
 0.0  -4.44089e-16   0.0        
 0.0   0.0           2.22045e-16
 0.0  -4.44089e-16  -4.44089e-16
 0.0  -2.22045e-16   0.0        

Bingo! It works!

## Watch the algorithm at work!

Copy and paste the code into <a href="http://edx-org-utaustinx.s3.amazonaws.com/UT501x/PictureFlame/PictureFLAME.html"> PictureFLAME </a>, a webpage where you can watch your routine in action.  Just cut and paste into the box.  

Disclaimer: we implemented a VERY simple interpreter.  If you do something wrong, we cannot guarantee the results.  But if you do it right, you are in for a treat.

If you want to reset the problem, just click in the box into which you pasted the code and hit "next" again.